In [24]:
import math
import time 
from typing import Any, Callable, Union, Optional

import numpy as np
import torch

from Evol_Instruct.MCTS.utils import extract_template, action_stop_words_mapping
from Evol_Instruct.models.vllm_support import get_vllm_model, vllm_clean_generate
from Evol_Instruct.prompts.prompt_template import mcts_prompts
from Evol_Instruct import client, logger

import pickle
import pdb
from Evol_Instruct.MCTS.tree import MCTS, MCTSConfig

In [25]:
model, lora_request = get_vllm_model(
    model_path='/mnt/petrelfs/jiangshuyang.p/models/Meta-Llama-3-8B-Instruct',
)
data = client.read("s3://syj_test/datasets/medical_train/mmed_en_train.json")
item = data[2]
logger.info(item)
logger.info(f"Correct answer: {item['answer_idx']}")

# # print(node)


INFO 11-11 10:53:11 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='/mnt/petrelfs/jiangshuyang.p/models/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='/mnt/petrelfs/jiangshuyang.p/models/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/mnt/petrelfs/jiangshuyang.p/model

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 11-11 10:53:38 model_runner.py:1025] Loading model weights took 14.9595 GB
INFO 11-11 10:53:40 gpu_executor.py:122] # GPU blocks: 27870, # CPU blocks: 2048
INFO 11-11 10:53:41 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-11 10:53:41 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-11 10:54:02 model_runner.py:1456] Graph capturing finished in 21 secs.
2024-11-11 10:54:02,641 - 71414734.py[line:6] - INFO - {'id': 'cmexam_2', 'conversations': [{'from': 'human', 'value': "A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He 

In [27]:
config = client.read("/mnt/petrelfs/jiangshuyang.p/repo/new/WizardLM/Evol_Instruct/config/trail1.json")
config = MCTSConfig(config)
tree = MCTS(item, model, model.get_tokenizer(), config=config)
# timing
start = time.time()
root = tree.run()
end = time.time()

logger.info(f"Time cost: {end - start}")
# with open("datas/tree2.pkl", 'wb') as file:
#     pickle.dump(root, file)
# with open("datas/tree2.pkl", 'rb') as file:
#     node = pickle.load(file)
# print_node(root)
# correct_path = root.obtain_correct_leaves()
node, reasoning_step = tree.inference()
print(reasoning_step)
print("*" * 80)
print(node.get_trajectory())

> /mnt/petrelfs/jiangshuyang.p/repo/new/WizardLM/Evol_Instruct/MCTS/tree.py(548)expand_node()
    546                                               **sampling_params) 
    547         pdb.set_trace()
--> 548         for i in range(len(prompts)):
    549             step = reasoning_steps[i][0]
    550             step = step.replace("<steps>", "").strip("\n")



KeyboardInterrupt: 